In [ ]:
# Необходимо для визуализации
# Required for visualization
!pip install pycocotools

In [ ]:
# Нужно для изменения размера bbox при изменении размера изображения
# Needed to resize bbox when resizing image
!pip install ../input/biblioteki/bibliot/albumentations-0.5.2

In [ ]:
import csv
import numpy as np
import json
import ast
import pandas as pd
import datetime
import cv2
import albumentations as A

In [ ]:
# Отбираем изображения, для которых имеются данные о координатах BBOX
# Selecting images for which there is data on BBOX coordinates
tr = pd.read_csv('../input/binary-csv/bincsv/train_binary.csv')
vl = pd.read_csv('../input/binary-csv/bincsv/val_binary.csv')
vl.loc[vl['id']=='2f12dbb2caf2']

In [ ]:
# For training with 4 classes
tr = tr[(tr['label_encoded']>0) & (tr['label']!='none 1 0 0 1 1')]
tr

In [ ]:
# For training with 2 classes
tr = tr[(tr['binary_label']>0) & (tr['label']!='none 1 0 0 1 1')]
tr

In [ ]:
tr.to_csv('./tr_last.csv', index=False)

In [ ]:
len(tr)

In [ ]:
# For training with 4 classes
vl = vl[(vl['label_encoded']>0) & (vl['label']!='none 1 0 0 1 1')]

In [ ]:
# For training with 2 classes
vl = vl[(vl['binary_label']>0) & (vl['label']!='none 1 0 0 1 1')]

In [ ]:
vl.to_csv('./vl_last.csv', index=False)

In [ ]:
len(vl)

In [ ]:
tr = pd.read_csv('./tr_last.csv')
vl = pd.read_csv('./vl_last.csv')
vl.loc[vl['id']=='2f12dbb2caf2']

In [ ]:
# Labels for classification
# Метки для классификации
labels = {
0:"Negative for Pneumonia",
1:"Typical Appearance",
2:"Indeterminate Appearance",
3:"Atypical Appearance"}

In [ ]:
# Labels for classification
# Метки для классификации
labels = {
0:"negative",
1:"opacity"}

In [ ]:
now = datetime.datetime.now()

data = dict(
    images=[
        # license, url, file_name, height, width, date_captured, id
    ],
    type='instances',
    annotations=[
        # segmentation, area, iscrowd, image_id, bbox, category_id, id
    ],
    categories=[
        # supercategory, id, name
    ],
)

In [ ]:
class_name_to_id = {}
for i, each_label in enumerate(labels):
    class_name = each_label
    class_id = i
    class_name_to_id[class_name] = class_id
    data['categories'].append(dict(
        supercategory=None,
        id=class_id,
        name=str(class_name),
    ))

In [ ]:
data

In [ ]:
# Определены следующие функции API:
# encode - Кодировать двоичные маски с помощью RLE.
# decode - декодировать двоичные маски, закодированные с помощью RLE.
# merge - вычислить объединение или пересечение закодированных масок.
# iou - вычислить пересечение по объединению масок.
# area - Расчетная область закодированных масок.
# toBbox - получить ограничивающие рамки, окружающие закодированные маски.
# frPyObjects - Преобразование многоугольника, bbox и несжатого RLE в закодированную маску RLE.

#  Rs     = encode( masks )
#  masks  = decode( Rs )
#  R      = merge( Rs, intersect=false )
#  o      = iou( dt, gt, iscrowd )
#  a      = area( Rs )
#  bbs    = toBbox( Rs )
#  Rs     = frPyObjects( [pyObjects], h, w )

# In the API the following formats are used:
#  Rs      - [dict] Run-length encoding of binary masks
#  R       - dict Run-length encoding of binary mask
#  masks   - [hxwxn] Binary mask(s) (must have type np.ndarray(dtype=uint8) in column-major order)
#  iscrowd - [nx1] list of np.ndarray. 1 indicates corresponding gt image has crowd region to ignore
#  bbs     - [nx4] Bounding box(es) stored as [x y w h]
#  poly    - Polygon stored as [[x1 y1 x2 y2...],[x1 y1 ...],...] (2D list)
#  dt,gt   - May be either bounding boxes or encoded masks
# Both poly and bbs are 0-indexed (bbox=[0 0 1 1] encloses first pixel).

In [ ]:
# JSON для изображений 512*512
# JSON for 512 * 512 images
train_out_file = './train_1280_annotations.json'
val_out_file = './val_1280_annotations.json'
data_val = data.copy()
data_val['images'] = []
data_val['annotations'] = []
data_train = data.copy()
data_train['images'] = []
data_train['annotations'] = []
# JSON для оригинальных изображений
# JSON original images
train_original_out_file = './train_annotations.json'
val_original_out_file = './val_annotations.json'
data_original_val = data.copy()
data_original_val['images'] = []
data_original_val['annotations'] = []
data_original_train = data.copy()
data_original_train['images'] = []
data_original_train['annotations'] = []

# JSON для измененных изображений
# JSON for modified images

In [ ]:
len(tr)

In [ ]:
im_size = 512

In [ ]:
# Трансформатор изображения и bbox
# Image transformer and bbox
transform = A.Compose(
    [
        A.Resize(height = im_size , width = im_size, p=1),
    ], 
    p=1.0,  bbox_params=A.BboxParams( format='coco', min_area=0,  min_visibility=0, label_fields=['labels']  ))        

In [ ]:
for idx in range(len(tr)):
    image_id = tr.iloc[idx].id
    class_id = tr.iloc[idx].label_encoded #binary_label
    path = tr.iloc[idx].path
    img = cv2.imread((path), cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#.astype(np.float32)
    width = im_size #int(tr.iloc[idx].w)
    height = im_size #int(tr.iloc[idx].h)
    data_train['images'].append(dict(
            file_name = image_id+'.png',
            width = width,
            height = height,
            date_captured=None,
            id=int(idx)))

    try:
        a = tr.iloc[idx].boxes
        boxes = ast.literal_eval(a)
        for box in boxes:
            x1= float(box['x'])
            y1 = float(box['y'])
            w = float(box['width'])
            h = float(box['height'])
            bb = []
            bbox = [
                    int(x1),
                    int(y1),
                    int(w),
                    int(h)]
            bb.append(bbox)
            a = int(class_id)
            b = str(class_id)
            ct = {f'{a}:{b}'}
            sample = transform(image=img, bboxes=bb, labels=b)
            tr_boxes = sample['bboxes']
            x_1 = tr_boxes[0][0]
            y_1 = tr_boxes[0][1]
            w_1 = tr_boxes[0][2]
            h_1 = tr_boxes[0][3]
            b_b = [int(x_1), int(y_1), int(w_1), int(h_1)]
            tr_labels = sample['labels']
            area = (w_1)*(h_1)
            data_train['annotations'].append(dict(id=len(data_train['annotations']),
                                                          area=round(area,3), 
                                                          bbox=b_b,
                                                          iscrowd= 0, #1,
                                                          image_id=int(idx),
                                                          category_id=int(tr_labels[0])))
                                                          #segmentation = rl ))
    except ValueError:
        print(image_id)
    
#     with open(train_out_file, 'w') as f:
#         json.dump(data_train, f, indent=4)

In [ ]:
data_train

In [ ]:
def myconverter(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime.datetime):
        return obj.__str__()

In [ ]:
with open(train_out_file, 'w') as f:
    json.dump(data_train, f, default=myconverter)#indent=4)

In [ ]:
for idx in range(len(vl)):
    image_id = vl.iloc[idx].id
    class_id = vl.iloc[idx].label_encoded #binary_label
    path = vl.iloc[idx].path
    img = cv2.imread((path), cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)#.astype(np.float32)
    width = im_size #int(vl.iloc[idx].w)
    height = im_size #int(vl.iloc[idx].h)
    data_val['images'].append(dict(
            file_name = image_id+'.png',
            width = width,
            height = height,
            date_captured=None,
            id=int(idx)))

    try:
        a = vl.iloc[idx].boxes
        boxes = ast.literal_eval(a)
        for box in boxes:
            x1= float(box['x'])
            y1 = float(box['y'])
            w = float(box['width'])
            h = float(box['height'])
            bb = []
            bbox = [
                    int(x1),
                    int(y1),
                    int(w),
                    int(h)]
            bb.append(bbox)
            a = int(class_id)
            b = str(class_id)
            ct = {f'{a}:{b}'}
            sample = transform(image=img, bboxes=bb, labels=b)
            vl_boxes = sample['bboxes']
            x_1 = vl_boxes[0][0]
            y_1 = vl_boxes[0][1]
            w_1 = vl_boxes[0][2]
            h_1 = vl_boxes[0][3]
            b_b = [int(x_1), int(y_1), int(w_1), int(h_1)]
            vl_labels = sample['labels']
            area = (w_1)*(h_1)
            data_val['annotations'].append(dict(id=len(data_val['annotations']),
                                                          area=round(area,3), 
                                                          bbox=b_b,
                                                          iscrowd= 0, #1,
                                                          image_id=int(idx),
                                                          category_id=int(vl_labels[0])))
                                                          #segmentation = rl ))
    except ValueError:
        print(image_id)
    
#     with open(train_out_file, 'w') as f:
#         json.dump(data_train, f, indent=4)

In [ ]:
data_val

In [ ]:
def myconverter(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime.datetime):
        return obj.__str__()

In [ ]:
with open(val_out_file, 'w') as f:
    json.dump(data_val, f, default=myconverter)#indent=4)

   # JSON для оригинальных изображений
   # JSON for original images

In [ ]:
len(tr)

In [ ]:
for idx in range(len(tr)):
    image_id = tr.iloc[idx].id
    class_id = tr.iloc[idx].label_encoded
    width = int(tr.iloc[idx].w)
    height = int(tr.iloc[idx].h)
    data_original_train['images'].append(dict(
            file_name = image_id+'.jpg',
            width = width,
            height = height,
            date_captured=None,
            id=int(idx)))

    try:
        a = tr.iloc[idx].boxes
        boxes = ast.literal_eval(a)
        for box in boxes:
            x1= float(box['x'])
            y1 = float(box['y'])
            w = float(box['width'])
            h = float(box['height'])
            bbox =[
                    x1,
                    y1,
                    w,
                    h]
            area = (w)*(h)
            data_original_train['annotations'].append(dict(id=len(data_original_train['annotations']),
                                                      area=area, 
                                                      bbox=bbox,
                                                      iscrowd= 0, #1,
                                                      image_id=int(idx),
                                                      category_id=int(class_id)))
                                                      #segmentation = rl ))
    except ValueError:
        print(image_id)
    
#     with open(train_out_file, 'w') as f:
#         json.dump(data_train, f, indent=4)

In [ ]:
data_original_train

In [ ]:
def myconverter(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime.datetime):
        return obj.__str__()

In [ ]:
with open(train_original_out_file, 'w') as f:
    json.dump(data_original_train, f, default=myconverter)#indent=4)

In [ ]:
len(vl)

In [ ]:
for idx in range(len(vl)):
    image_id = vl.iloc[idx].id
    class_id = vl.iloc[idx].label_encoded
    width = int(vl.iloc[idx].w)
    height = int(vl.iloc[idx].h)
    data_original_val['images'].append(dict(
            file_name = image_id+'.jpg',
            width = width,
            height = height,
            date_captured=None,
            id=int(idx)))

    try:
        a = vl.iloc[idx].boxes
        boxes = ast.literal_eval(a)
        for box in boxes:
            x1= float(box['x'])
            y1 = float(box['y'])
            w = float(box['width'])
            h = float(box['height'])
            bbox =[
                    x1,
                    y1,
                    w,
                    h]
            area = (w)*(h)
            data_original_val['annotations'].append(dict(id=len(data_original_val['annotations']),
                                                      area=area, 
                                                      bbox=bbox,
                                                      iscrowd= 0, #1,
                                                      image_id=int(idx),
                                                      category_id=int(class_id)))
                                                      #segmentation = rl ))
    except ValueError:
        print(image_id)
    
#     with open(train_out_file, 'w') as f:
#         json.dump(data_train, f, indent=4)

In [ ]:
data_original_val

In [ ]:
def myconverter(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, datetime.datetime):
        return obj.__str__()

In [ ]:
with open(val_original_out_file, 'w') as f:
    json.dump(data_original_val, f, default=myconverter)#indent=4)

In [ ]:
# Визуализация
# Visualization
from pycocotools.coco import COCO
import cv2
from matplotlib import pyplot as plt

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

In [ ]:
def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
json_file = './train_512_annotations.json'

In [ ]:
font = cv2.FONT_HERSHEY_PLAIN
coco=COCO(json_file)
cats = coco.loadCats(coco.getCatIds())
ids = list(sorted(coco.imgs.keys()))
img_id = ids[617]
ann_ids = coco.getAnnIds(imgIds=img_id) # номера аннотаций изображения с боксами и пр
coco_annotation = coco.loadAnns(ann_ids)
path = coco.loadImgs(img_id)[0]['file_name']
classes = [obj["category_id"] for obj in coco_annotation]

img = cv2.imread(f'../input/coco-512/siim_512/train_512/{path}', cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
num_objs = len(coco_annotation)
boxes = [] # список боксов в формате вок х1,у1,х2,у2
cats = coco.loadCats(classes)# Найти категорию по идентификатору категории
c = []
d = []
for cat in cats:
    a = cat['id']
    b = cat['name']
    c.append(a)
    d.append(b)
ct = dict(zip(c, d))
for i in range(num_objs):
    xmin = coco_annotation[i]['bbox'][0]-1
    ymin = coco_annotation[i]['bbox'][1]-1
    xmax = coco_annotation[i]['bbox'][2]-1
    ymax = coco_annotation[i]['bbox'][3]-1
    box = [xmin, ymin, xmax, ymax]
    boxes.append(box)
visualize(img, boxes, classes, ct)